# ADA Project : Milestone 2

## Imports and Data

In [1]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sn

In [2]:
# Data from RateBeer 'RB'
beers_RB = pd.read_table("./data/RateBeer/beers.csv", sep=",")
breweries_RB = pd.read_table("./data/RateBeer/breweries.csv", sep=",")
users_RB = pd.read_table("./data/RateBeer/users.csv", sep=",")
ratings_RB = pd.read_table("./data/RateBeer/ratings.csv", sep=",")

In [3]:
# Data from BeerAdvocate 'BA'
beers_BA = pd.read_table("./data/BeerAdvocate/beers.csv", sep=",")
breweries_BA = pd.read_table("./data/BeerAdvocate/breweries.csv", sep=",")
users_BA = pd.read_table("./data/BeerAdvocate/users.csv", sep=",")
ratings_BA = pd.read_table("./data/BeerAdvocate/ratings.csv", sep=",")

In [4]:
# Data from MixedDataset 'MD'
beers_MD = pd.read_table("./data/matched_beer_data/beers.csv", sep=",")
breweries_MD = pd.read_table("./data/matched_beer_data/breweries.csv", sep=",")
users_MD = pd.read_table("./data/matched_beer_data/users.csv", sep=",")
users_approx_MD = pd.read_table("./data/matched_beer_data/users_approx.csv", sep=",")
ratings_MD = pd.read_table("./data/matched_beer_data/ratings.csv", sep=",")

FileNotFoundError: [Errno 2] No such file or directory: './data/matched_beaer_data/beers.csv'

## Seasonal analysis of the number of reviews per type of beer

In [ ]:
import datetime as dt

# create sub_dataframe with type of beer and date
df_count_type_season = df[['type','date']].dropna()

# create extra column with the day in year corresponding to the date
df_count_type_season['day_of_year'] = df_count_type_season['date'].dt.dayofyear

num_bins = 10
bins = 365/num_bins * np.range(1, num_bins + 1)
groups = df.groupby(['type', pd.cut(df.day_of_year, bins)])
print(groups.size().unstack())
groups.plot(type='line')

exit()

# for loop on the different type of beer
types = pd.unique(df_count_type_season['type'])

plt.figure(figsize=(5,5))
for beertype in types:
    df_count = df_count_type_season[df_count_type_season['type'] == beertype][day_of_year].group_by('day_of_year').count()
    print(df_count.iloc[:5])
    
    # count per day (or broader bins)
    # plot

## Analysis 2

## Analysis 3

## Analysis 4